# Loops, Function Arguments, and Discrete-Time Dynamical Systems

Today, we are going to go over for- and while-loops, talk a little more about function arguments and passing data to functions, and simulate a discrete-time dynamical system as a motivating example.

## For Loops

For-loops iterate over a specified set of values, running the code within the for-loop block once for each value in the set.

Often, we just want to repeat a given task a certain number of times, say N times. We can do this by iterating over the numbers 0 through N-1 as a counter for which repetition we are on. The built-in `range(N)` function returns an iterable of length N corresponding to the numbers from 0 to N-1.

The syntax for a for-loop is as follows. Note the colon and the indentation. *Python begins and ends code blocks based on identation, which much be consistent throughout the block*. Normal indentation is four spaces, which you should automatically get in VS Code whenever you hit the tab key.

In [ ]:
for n in range(5):
    print('The current value of n is: ')
    print(n)
print('The for-loop is done.')

The full syntax for the range function is `range(start,stop,step)`. If only one number is supplied, it is interpreted as the stop value (which is excluded, just like in slices), and the step is 1. If two numbers are supplied, step is assumed to be 1. Again, just like with slices, start is included but stop is excluded. 

Finally, you can supply different values for step, but they must be non-zero integer values. Decimal values will return a TypeError.

Try some different things with range below!

`range` is a special kind of data type in Python, built for speed because it is used in for-loops so much. So if you try to print it, you won't get a sequence of numbers but a generator-type object. However, you can use it to generate lists or tuples by calling those functions on it.

In [ ]:
print(range(5))
tup_range = tuple(range(5))
print(tup_range)

You can also create lists/tuples via number ranges using a method called *list comprehension*. It's essentially a mini for-loop inside of list or tuple brackets. For example, suppose we want to put together a list of perfect squares from 1 to 10:

In [ ]:
squares = [n**2 for n in range(1, 11)] # remember 11 is not included
print(squares)

However, for-loops in Python work over far more than just range objects. You can loop over any "iterable" type data structure. That includes lists, tuples, and numpy arrays:

In [ ]:
for word in ('way','to','go!'):
    print(word)

Ok, but what if we want a counter as we loop through a list of items? That's when the built-in `enumerate` function comes in handy. This is powerful for grabbing some data from a data structure, doing something with it, and then storing it in another data structure in the proper order.

In [ ]:
for n, item in enumerate(['blue', 'green', 'yellow']):
    print(f'item number {n} is {item}')

Finally, if your for-loop consists of only one line (like many of the ones above), you can just place that one line after the colon instead of indenting it below.

In [ ]:
L = [2,5,6,7,9]
for x in L: print(x)

For-loops (and code blocks in general) can also be *nested*.

In [ ]:
for x in range(4):
    for y in range(1,3):
        print(f'x is {x} and y is {y}')
        print(f'x^y is {x**y}')
    print('That\'s all for this value of x.')
print('The for-loop is done.')

## While Loops

Many times, instead of looping over a fixed set, we want to perform some sort of task until a condition is met (like a convergence condition in a numerical solver). This can be accomplished with a while-loop.

In [ ]:
x = 0
while x <= 4:
    print(x)
    x += 1
print('The while-loop is done.')

You have to be careful with while-loops, because it is possible for your code to get stuck in an infinite loop if the condition is never satisfied or will take an increadibly long time to be satisfied. In this case, your code may simply continue to run indefinitely, or your program could crash or your computer freeze when it runs out of memory (if you have a *memory leak*, which is where inside your while loop, you are creating more and more data in memory). 

Note: you can usually force-stop running code by holding the control button and hitting the C key, but sometimes you have to fdo other things instead.

To prevent all of this, it may be a good idea to include a break command which will exit the while-loop if a condition is met. You can specify this condition using an if-statement block. We'll talk more about these later.

In [ ]:
x = 0
while x <= 100:
    print(x)
    x += 1
    if x > 10:
        break
print('The while-loop is done.')

## Function arguments

Often you will want to call one or more functions inside of a for- or while-loop, and in a variety of other circumstances, so let's go over some more syntax for calling functions.

Suppose we have that logistic ODE we were working with last time. Let's add an Allee effect (you can learn about this in Math 581) and pass the parameter values as function arguments:

In [ ]:
def allee_ode(t, N, r, K, K0):
    """Calculates the derivative of x with respect to time using the logistic growth model."""
    return r*N*(1 - N/K)*(N/K0 - 1)

r = 0.1
K = 100
K0 = 10
print(allee_ode(0, 50, r, K, K0))

As the number of parameters gets long, it could be pretty cumbersome to specify each one individually like this in a function call. Another option is to put them all in a tuple and then unpack it into the function call:

In [ ]:
args = (0, 50, r, K, K0)
print(allee_ode(*args)) # The asterisk unpacks the tuple into separate arguments for the function.

A better option for passing parameters is to use a dictionary. Dictionaries are great because the keys for the dictionary can be strings, which can be greek letters or descriptors that match what you have in your research writeup.

In [ ]:
params = {}
params['r'] = 0.1
params['K'] = 100
params['K0'] = 10
print(allee_ode(0, 50, **params)) # The double asterisk unpacks the dictionary into separate keyword arguments for the function.

But maybe you plan on keeping one or more of these parameters fixed most of the time - you may want to make it a default value, so you don't have to specify it at all.

Default values for function arguments are specfied when you define the function, and once you start specifying default values, all parameters to the right must also have default values.

In [ ]:
def allee_ode(t, N, r, K=100, K0=10):
    """Calculates the derivative of x with respect to time using the logistic growth model."""
    return r*N*(1 - N/K)*(N/K0 - 1)

r = 0.1
K = 100
K0 = 10
print(allee_ode(0, 50, r)) # K and K0 will use their default values of 100 and 10, respectively.

With a large number of parameters, all of this can still result in an enormous function definition. Suppose you have 10-20 parameters. You are going to end up writing that function call signature on multiple lines, and that's going to get hard to read quickly!

A better solution is to pass in all the parameters at once - e.g., using that dictionary again. But what if you want default values in case a dictionary is NOT passed in? Well, you could specify a default dictionary, but this brings up an important rule:

**DEFAULT VALUES SHOULD NEVER BE MUTABLE OBJECTS!** This includes numpy arrays.

The reason why is somewhat complex, but trust me: if you call the function multiple times, things will start to go south in a hurry.

So, what can you do? The Python idiom for this is to use a default value of `None`. `None` is a special keyword datatype in Python that points to nothing at all. It's unique and immutable. It's usage can go something like this:

In [ ]:
def allee_ode(t, N, params=None):
    """Calculates the derivative of x with respect to time using the logistic growth model."""

    if params is None: # this is how you check to see if a variable has been assigned a value of None
        params = {} # empty dictionary, so that the get method can be used below without raising an error 
                    #     if params is not provided as an argument when the function is called.
    
    # the get method of a dictionary returns the value for the specified key if the key is in the dictionary, 
    #    just like indexing with square brackets, but if the key is not found in the dictionary,
    #    it can return a default value provided as the second argument (or None if no default value is provided).
    #    Trying to access a key that does not exist in the dictionary using square brackets would raise a KeyError instead.
    r = params.get('r', 0.1)  # default value of r is 0.1
    K = params.get('K', 100)  # default value of K is 100
    K0 = params.get('K0', 10)  # default value of K0 is 10

    return r*N*(1 - N/K)*(N/K0 - 1)

print(allee_ode(0, 50)) # K and K0 will use their default values of 100 and 10, respectively.

Try creating a dictionary and passing it to the function call in the last line of the code block below to see how that works too, even if you don't specify all of the parameters!

## Discrete-time Dynamical Systems

Let's use what we have learned above to analyze the logistic map, a discrete-time population model that you can learn about in Math 581.

The logistic map can be expressed via the equation
$$x_{t+1} = x_t + Rx_t\left(1-\frac{x_t}{K}\right)$$
where $R$ is the discrete-time growth rate and $K$ is the carrying capacity. Given the size of the population at a time $t\in\mathbb{N}$, this equation gives you the size of the population at the next time, $t+1$. Paired with an initial condition $x_0$, it can be iteratively solved for a sequence of population values at all positive integer times $t$.

The right-hand side of this equation, call it $f(x_t)$, is referred to as the "updating function" because it updates $x_t$ to the next time $t$. 

In the cell below, create a function called `log_map` that implements the updating function with arguments x, r, and K in that order. You don't need to include $t$ because there is no explicit dependence on $t$ in the updating function (it is an autonomous difference equation) and we won't be using `solve_ivp` because it's not an ODE. Since there are only two parameters, you don't need to use a dictionary, but let's go ahead and set default values of $R=1$ and $K=100$.

Now we can call the updating function in a loop to simulate the model. Let's start with an initial population of 4 and simulate 20 time-steps. Because we might change the number of time-steps later on, we will create a variable for the number of steps we want to run and append each result as it comes in to a list:

In [ ]:
pop_list = [4]
N = 20 # number of time steps
for t in range(N):
    pop_list.append(log_map(pop_list[-1])) # take the last element of pop_list, apply the log_map function to it with default parameter values, 
                                           #    and append the result to pop_list

print(pop_list) # Note that this contains N+1 elements: the initial condition at t=0 and the solutions through to t=N.

Plotting the result will give us a better picture of what is happening.

In [ ]:
# Plot the result of the logistic map
import matplotlib.pyplot as plt

pop_list = [4]
N = 20 # number of time steps
for t in range(N):
    pop_list.append(log_map(pop_list[-1])) # take the last element of pop_list, apply the log_map function to it with default parameter values, 
                                           #    and append the result to pop_list

plt.figure(figsize=(10,6)) # set the size of the plot (here making it bigger than default)
plt.plot(pop_list, marker='o') # If you exclude x-values, they are assumed to be integers starting with 0
                               # Also, we can include markers to show the individual points on the plot
plt.xlabel('Time step')
plt.ylabel('Population')
plt.title('Logistic Map')
plt.show()

Now in the above code, try changing the value of R so that we can see some of the behavior discussed in Math 581. $R=1.8$ should give you a damped oscillation. $R=2.2$ should give you an asymptotically stable 2-cycle. $R=2.5$ should give you an asymptotically stable 4-cycle. $R=2.55$ should give you an asymptotically stable 8-cycle (look carefully). $R=3$ is chaotic. Note that you can try increasing the number of time steps to 80 to see some of these dynamics better.

Now let's try to recreate the bifurcation diagram from Kot which summarizes these dynamics for $1.8\leq r\leq 3$. This involves sweeping through the bifurcation parameter $r$. For each value of $r$, we will run the model for a while (to try and neglect transient behavior in favor of the asymptotics as $t$ gets large) and then save the later values to plot. If the dynamics are asymptotic to a single value, that value will just get plotted on top of itself over and over again. If it's asymptotic to a 2-cycle, those two values will get plotted over and over again, etc. If the dynamics are chaotic, the plotting will fill out a region of space instead.

We then do this for each $r$, and make a diagram with $x_t$ values we recorded on the y-axis and $r$ values on the x-axis. Like this:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

burn_in = 150 # number of initial time steps to discard (to allow the system to settle into its long-term behavior)
N = 300 # total number of time steps to run the logistic map (including burn-in)
r_vals = np.linspace(1.8, 3.0, 200) # 100 values of r evenly spaced between 2.8 and 3.0

x_vals = [] # x-values for the plot
pop_vals = [] # this will hold all our data.
for r in r_vals:
    x = 4 # initial condition
    for t in range(burn_in):
        x = log_map(x, r) # update x using the logistic map with the current value of r
    for t in range(N - burn_in):
        x = log_map(x, r) # update x using the logistic map with the current value of r
        x_vals.append(r) # append the current value of r to the list of x-values for the plot
        pop_vals.append(x) # append the current population value to the list of population values for the plot

plt.figure(figsize=(10,6))
plt.scatter(x_vals, pop_vals, s=0.15) # use a scatter plot to show the relationship between r and the population values
plt.xlabel('Growth rate (r)')
plt.ylabel('Population')
plt.title('Bifurcation Diagram for the Logistic Map')
plt.show()

Play with the above code! Try refining it (without making your computer chew on the computation forever), and then try zooming in on areas of the bifurcation diagram by adjusting your R ranges. For instance, you can explore the windows where asymptotic cycles reappear!